In [ ]:
import os

from google import genai
from google.genai import types

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

response = client.models.generate_content(
    # model='gemini-1.5-flash',
    model="gemini-2.0-flash",
    contents="Who won Wimbledon this year?",
    config=types.GenerateContentConfig(tools=[types.Tool(google_search=types.GoogleSearchRetrieval)]),
)
print(response)

In [ ]:
from google.genai.types import FunctionDeclaration, Schema, Tool

weather_tool = Tool(
    function_declarations=[
        FunctionDeclaration(
            name="get_weather",
            description="Fetches the current weather for a given city.",
            parameters=Schema(
                type="object",
                properties={"city": {"type": "string", "description": "The city to get the weather for."}},
                required=["city"],
            ),
        )
    ]
)

import requests


def get_weather(city):
    # Mock API URL (replace with a real weather API)
    api_url = f"https://api.weatherapi.com/v1/current.json?key=YOUR_API_KEY&q={city}"

    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        return f"The weather in {city} is {data['current']['condition']['text']} with a temperature of {data['current']['temp_c']}°C."
    else:
        return "Sorry, I couldn't fetch the weather."


from google.genai.types import GenerateContentConfig, GoogleSearch

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

# tool_config=FunctionCallingConfig(mode="ANY")

# Define tools: Google Search + Custom Weather Tool
google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="What's the weather like in New York and the latest AI news?",
    config=GenerateContentConfig(
        tools=[{"google_search": {}}, weather_tool],
    ),
)

# Process the response
for part in response.candidates[0].content.parts:
    print(part.text)